<a href="https://colab.research.google.com/github/ml2-picme/PicMe/blob/master/Image%20Download%20and%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Helper Functions

In [0]:
from keras.preprocessing.image import load_img, img_to_array
import urllib.request

def downloadFileFromUrl(URL, localPath):
  print("Lade ", URL, " nach ", localPath + " herunter")
  with urllib.request.urlopen(URL) as url:
    with open(localPath, 'wb') as f:
      f.write(url.read())
  load_img(localPath)

In [0]:
import os

def createLocalDirectory(dir):
  
  print("Erstelle lokales Verzeichnis:", dir, end='\t')
  
  try:  
    os.makedirs(dir)

  except OSError:  
    print (" - Failed")
  else:
    print(" - OK")

In [0]:
import os
import shutil

def deleteLocalFiles(dir):
  try:
    print("Delete local files", end='')  
    shutil.rmtree(dir) 
    print(" - OK")
  except OSError:
    print(" - Failed")

In [0]:
import os

def findFilesInPath(dir):

  files = []
  
  # r=root, d=directories, f = files
  for r, d, f in os.walk(dir):
    for file in f:
        if file.endswith(".jpg") or file.endswith(".png"):
            files.append(os.path.join(r, file))
            
  return files

In [0]:
from PIL import Image
import numpy as np

def prepareImagesForClassification(files, sizeX, sizeY):
  images = []
  
  for file in files:
    try:
      image = Image.open(file)
      image = image.resize((sizeX, sizeY), Image.LANCZOS)
      image = image.convert("RGB")
      
      image = np.asarray(image)
      images.append(image)
    except OSError:
      pass
  
  images = np.asarray(images)
  return images

In [0]:
def classifyImages(preparedImages, preprocess_input_function, decode_predictions_function, model):
  
  # preprocess the images to fit to the model
  images_preprocessed = preprocess_input_function(preparedImages)
  
  # use the model to classifi the images
  images_pred = model.predict(images_preprocessed, verbose=1)
  
  pred_results = decode_predictions_function(images_pred)
  
  return pred_results

In [0]:
import matplotlib.pyplot as plt

def printClassificationResults(images, predictedClasses):
  
  for i in range(len(images)):
    plt.figure()
    plt.imshow(images[i])
    plt.title(predictedClasses[i])
    plt.axis('off')
    
  plt.show()

In [0]:
def compareResults(fileNames, resulsList, modelList, threshold):
  
  for i in range(len(fileNames)):
    print("Comparing the Results for File: " + fileNames[i])
    
    for j in range(5):
      for k in range(len(modelList)):
        resultToPrint = resultsList[k][i][j]
        
        # Setze 5% als Threshold
        if(resultToPrint is not None and resultToPrint[2] > threshold):
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", resultsList[k][i][j])
        else:
          print(" > ", (j+1), ". Platz @ ", modelList[k], " : ", "--- Threshold-Filter ---")
      
      print("=================================================================")
      
    print("")

## Logic

In [0]:
# Parameters
path = "/tmp/image_classification"
hashrange = 20

In [0]:
# Preparation: Delete local files
deleteLocalFiles(path)

In [0]:
# Preparation: Create local directory structure
for i in range(hashrange):
  if(i % 10 == 0):
    parentPath = path + "/" + str((int)(i/10))
    createLocalDirectory(parentPath)
  normalizedI = '%02d' % i  # Normalization, pad zeroes
  filePath = parentPath + "/" + normalizedI
  createLocalDirectory(filePath)

In [0]:
from urllib.request import urlopen

data = urlopen("https://raw.githubusercontent.com/ml2-picme/PicMe/master/prototyping/test_textfile.txt")
for line in data:
  line = line.decode("utf-8").split("\n")[0]  # Normalization
  url = line.split(";")[0]
  label = line.split(";")[1]
  
  filename = url.split("/")[len(url.split("/")) - 1]
  
  hashvalue = abs(hash(filename)) % hashrange
  parent_dir = (int)(hashvalue / 10)
  hashvalue = '%02d' % hashvalue  # Normalization, pad zeroes
  
  print(url, " -> ", hashvalue, " -> ", label, " -> ", parent_dir, " -> ", filename)
  
  localPath = path + "/" + str(parent_dir) + "/" + hashvalue + "/" + filename
  
  downloadFileFromUrl(url, localPath)

In [0]:
foundFiles = findFilesInPath(path)
preparedImages = prepareImagesForClassification(foundFiles, 224, 224)

In [0]:
from keras.applications import *

# Achtung: Hier werden Funktionen übergeben: 
# 1) preprocess_input Funktion
# 2) decode_predictions Funktion
# => Dies sorgt dafür, dass die gesamte Klassifizierung ausgelagert und dynmaisch aufgerufen werden kann!

predictedClassesVGG16 = classifyImages(preparedImages, vgg16.preprocess_input, vgg16.decode_predictions, vgg16.VGG16(input_shape=(224, 224, 3)))
predictedClassesVGG19 = classifyImages(preparedImages, vgg19.preprocess_input, vgg19.decode_predictions, vgg16.VGG16(input_shape=(224, 224, 3)))

In [0]:
# Optional: Print the images with the predicted classes
printClassificationResults(preparedImages, predictedClassesVGG16)
printClassificationResults(preparedImages, predictedClassesVGG19)

In [0]:
resultsList = [predictedClassesVGG16, predictedClassesVGG19]
modelList = ['VGG16', 'VGG19']

# Diese compareResults Methode braucht 4 Parameter:
# 1. Die Liste der Dateinamen
# 2. Die Ergebnisse der einzelnen Modellen, als Array zusammengefasst
# 3. Die Namen der Modelle, einfach als String Array
# 4. Threshold (mind. Sicherheit der Modellvorhersage)
compareResults(foundFiles, resultsList, modelList, 0.05)